In [51]:
!pip install spacy
!python -m spacy download en_core_web_sm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
import spacy
import tensorflow as tf
import tqdm
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 82.7 MB/s eta 0:00:00m eta 0:00:010:0101
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [247]:
# Load your dataset
# Replace 'your_dataset.csv' with the actual path or URL of your dataset
nlp = spacy.load("en_core_web_sm")
df = pd.read_csv('spam.csv', encoding = "ISO-8859-1")
df = df[['v1','v2']]


def preprocess_text(text):
    
    text = text.lower()
    text = re.sub("\d+", "", text)
    text = re.sub(r"[!\"#$%&()*+,-./:;<=>?@[\\\]^_`{|}~]+", "",text )
    text = re.sub(r"\s+", " ", text)
    doc = nlp(text)
    tokens = [token for token in doc]
    
    return tokens


# Preprocess the data
df['v2_tokens'] = df['v2'].apply(preprocess_text)
print(df)
X = df['v2_tokens']
y = df['v1']
le = LabelEncoder()








        v1                                                 v2  \
0      ham  Go until jurong point, crazy.. Available only ...   
1      ham                      Ok lar... Joking wif u oni...   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...   
3      ham  U dun say so early hor... U c already then say...   
4      ham  Nah I don't think he goes to usf, he lives aro...   
...    ...                                                ...   
5567  spam  This is the 2nd time we have tried 2 contact u...   
5568   ham              Will Ì_ b going to esplanade fr home?   
5569   ham  Pity, * was in mood for that. So...any other s...   
5570   ham  The guy did some bitching but I acted like i'd...   
5571   ham                         Rofl. Its true to its name   

                                              v2_tokens  
0     [go, until, jurong, point, crazy, available, o...  
1                        [ok, lar, joking, wif, u, oni]  
2     [free, entry, in, a, wkly, comp, to, wi

In [248]:
# Tokenize and encode the text using Word2Vec
word2vec_model = Word2Vec(sentences=X, vector_size=100, window=7, min_count=1, workers=-1)
words = set(word2vec_model.wv.index_to_key)


In [249]:
X_encoded = np.array([np.mean([word_vectors[word] for word in text
                               if word in word_vectors] or [np.zeros(100)], axis=0) for text in X])


In [254]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)
# Create a simple neural network with embedding layers
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=100)) 
model.add(Dense(32, activation='relu'))  
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")
    

Epoch 1/10
244/244 [==============================] - 2s 5ms/step - loss: 0.6542 - accuracy: 0.8646 - val_loss: 0.6163 - val_accuracy: 0.8690
Epoch 2/10
244/244 [==============================] - 1s 5ms/step - loss: 0.5863 - accuracy: 0.8646 - val_loss: 0.5559 - val_accuracy: 0.8690
Epoch 3/10
244/244 [==============================] - 1s 5ms/step - loss: 0.5345 - accuracy: 0.8646 - val_loss: 0.5104 - val_accuracy: 0.8690
Epoch 4/10
244/244 [==============================] - 1s 5ms/step - loss: 0.4958 - accuracy: 0.8646 - val_loss: 0.4764 - val_accuracy: 0.8690
Epoch 5/10
244/244 [==============================] - 1s 5ms/step - loss: 0.4671 - accuracy: 0.8646 - val_loss: 0.4512 - val_accuracy: 0.8690
Epoch 6/10
244/244 [==============================] - 1s 5ms/step - loss: 0.4461 - accuracy: 0.8646 - val_loss: 0.4329 - val_accuracy: 0.8690
Epoch 7/10
244/244 [==============================] - 1s 4ms/step - loss: 0.4309 - accuracy: 0.8646 - val_loss: 0.4194 - val_accuracy: 0.8690
Epoch 

In [255]:
y_pred = model.predict(X_test)
y_pred = [0 if pred[0] <0.5 else 1 for pred in y_pred]


53/53 [==============================] - 0s 430us/step


In [256]:
from sklearn.metrics import precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
accuracy = (y_pred==y_test).sum()/len(y_pred)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print('Accuracy: {} , Precision: {} , Recall: {}'.format(accuracy, precision, recall))
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.8690191387559809 , Precision: 0.0 , Recall: 0.0
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      1453
           1       0.00      0.00      0.00       219

    accuracy                           0.87      1672
   macro avg       0.43      0.50      0.46      1672
weighted avg       0.76      0.87      0.81      1672

[[1453    0]
 [ 219    0]]


/home/reda/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/reda/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/reda/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/reda/anaconda3/lib/python3.11/site-packages/sklea

In [230]:
import numpy as np

def clean_text(text):

    text = text.lower()
    text = re.sub("\d+", "", text)
    text = re.sub(r"[!\"#$%&()*+,-./:;<=>?@[\\\]^_`{|}~]+", "",text )
    text = re.sub(r"\s+", " ", text)
    return text
    
df['sms_clean'] = df['v2'].apply(clean_text)   
display(df)
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="out_of_vocab") # instanciate the tokenizer
tokenizer.fit_on_texts(df.sms_clean)
df["sms_encoded"] = tokenizer.texts_to_sequences(df.sms_clean)

,v1,v2,v2_tokens,sms_clean
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, crazy, available, o...",go until jurong point crazy available only in ...
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]",ok lar joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, a, wkly, comp, to, win, fa, ...",free entry in a wkly comp to win fa cup final ...
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, u, c, already, t...",u dun say so early hor u c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, do, n't, think, he, goes, to, usf, he...",nah i don't think he goes to usf he lives arou...
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[this, is, the, nd, time, we, have, tried, con...",this is the nd time we have tried contact u u ...
5568,ham,Will Ì_ b going to esplanade fr home?,"[will, ì, b, going, to, esplanade, fr, home]",will ì b going to esplanade fr home
5569,ham,"Pity, * was in mood for that. So...any other s...","[pity, was, in, mood, for, that, soany, other,...",pity was in mood for that soany other suggestions
5570,ham,The guy did some bitching but I acted like i'd...,"[the, guy, did, some, bitching, but, i, acted,...",the guy did some bitching but i acted like i'd...


In [231]:
sms_pad = tf.keras.preprocessing.sequence.pad_sequences(df.sms_encoded, padding="post")
le = LabelEncoder()
x_train, x_val, y_train, y_val = train_test_split(sms_pad,df['v1'], test_size=0.3)
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)
train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
val = tf.data.Dataset.from_tensor_slices((x_val, y_val))

train_batch = train.shuffle(len(train)).batch(16)
val_batch = val.shuffle(len(val)).batch(16)

In [233]:
sms_pad[0]

array([ 43, 441,   1, 784, 705, 675,  61,   9,   1,  88, 121, 351,   1,
       151,   1,   1,  65,  55,   1, 137,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0], dtype=int32)

In [234]:
import tensorflow as tf
from keras.layers import Dense,LSTM, Embedding, Dropout, Activation, Bidirectional

embedding_vector_length = 32

model = tf.keras.Sequential()
model.add(Embedding(1000, embedding_vector_length, input_length=171))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 171, 32)           32000     
                                                                 
 bidirectional_3 (Bidirecti  (None, 128)               49664     
 onal)                                                           
                                                                 
 dense_40 (Dense)            (None, 16)                2064      
                                                                 
 dropout_6 (Dropout)         (None, 16)                0         
                                                                 
 dense_41 (Dense)            (None, 1)                 17        
                                                                 
Total params: 83745 (327.13 KB)
Trainable params: 83745 (327.13 KB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [235]:
history = model.fit(train_batch, 
                    epochs=10, 
                    validation_data=val_batch)

Epoch 1/10
244/244 [==============================] - 8s 28ms/step - loss: 0.2150 - accuracy: 0.9323 - val_loss: 0.0746 - val_accuracy: 0.9767
Epoch 2/10
244/244 [==============================] - 5s 22ms/step - loss: 0.0558 - accuracy: 0.9854 - val_loss: 0.0877 - val_accuracy: 0.9767
Epoch 3/10
244/244 [==============================] - 5s 22ms/step - loss: 0.0435 - accuracy: 0.9897 - val_loss: 0.0873 - val_accuracy: 0.9809
Epoch 4/10
244/244 [==============================] - 5s 22ms/step - loss: 0.0331 - accuracy: 0.9910 - val_loss: 0.0741 - val_accuracy: 0.9791
Epoch 5/10
244/244 [==============================] - 6s 24ms/step - loss: 0.0248 - accuracy: 0.9949 - val_loss: 0.1009 - val_accuracy: 0.9833
Epoch 6/10
244/244 [==============================] - 6s 23ms/step - loss: 0.0272 - accuracy: 0.9928 - val_loss: 0.0915 - val_accuracy: 0.9767
Epoch 7/10
244/244 [==============================] - 5s 22ms/step - loss: 0.0181 - accuracy: 0.9951 - val_loss: 0.1020 - val_accuracy: 0.9815

In [244]:
y_pred = model.predict(x_val)
y_pred = [0 if pred[0] <0.5 else 1 for pred in y_pred]

53/53 [==============================] - 0s 5ms/step


In [246]:
from sklearn.metrics import precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
accuracy = (y_pred==y_val).sum()/len(y_pred)

precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)

print('Accuracy: {} , Precision: {} , Recall: {}'.format(accuracy, precision, recall))
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred))

Accuracy: 0.9784688995215312 , Precision: 0.9234449760765551 , Recall: 0.9061032863849765
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1459
           1       0.92      0.91      0.91       213

    accuracy                           0.98      1672
   macro avg       0.95      0.95      0.95      1672
weighted avg       0.98      0.98      0.98      1672

[[1443   16]
 [  20  193]]
